## Codice 1 OverpassTurbo (serve per tutti i quartieri tranne Forze Armate)

## Codice 2 Overpass Turbo (serve per il quartiere Forze Armate)

# Import del file .geojson scaricato da overpass

In [22]:
import geopandas as gpd

# Load the GeoJSON file
gdf_quarters = gpd.read_file("C:/Users/edoar/Downloads/export.geojson")

# Inspect the data
print(gdf_quarters.head())

              id            @id boundary               description  \
0  way/404140123  way/404140123   census  Nuclei d'Identità Locale   
1  way/404140124  way/404140124   census  Nuclei d'Identità Locale   
2  way/404140125  way/404140125   census  Nuclei d'Identità Locale   
3  way/404140126  way/404140126   census  Nuclei d'Identità Locale   
4  way/404140127  way/404140127   census  Nuclei d'Identità Locale   

                  name    place ref            source tourism wikidata  \
0  Parco delle Abbazie  quarter  85  Comune di Milano    None     None   
1              Adriano  quarter  17  Comune di Milano    None     None   
2               Affori  quarter  80  Comune di Milano    None     None   
3               Baggio  quarter  55              None    None     None   
4           Bande Nere  quarter  52  Comune di Milano    None     None   

  wikimedia_commons wikipedia  \
0              None      None   
1              None      None   
2              None      None   
3 

In [23]:
print(gdf_quarters.crs)  # Check CRS

EPSG:4326


# API di YELP

In [25]:
import requests
import pandas as pd
import time
import geopandas as gpd
from shapely.geometry import Point

# List of neighborhoods
NEIGHBORHOODS = [
    "Brera, Milan",
    "Duomo, Milan",
    "Porta Sempione, Milan",
    "Guastalla, Milan",
    # Add more neighborhoods as needed
]

# Replace with your Yelp API key
API_KEY = "NjJTmQHwjOzNCKyGBxPPj7RRAZHQIdYR5HbsKBUbMansLpfum3tnUvBIqZBX45_H2PMr5AhG1XBQn1PbIK0xpwNVst70cfG_ktE8WkRCHVAUGHonYlGrKH44ZQZWZ3Yx"  # Replace with your API key
HEADERS = {"Authorization": f"Bearer {API_KEY}"}

# Constants
BUSINESSES_PER_REQUEST = 48  # Maximum allowed by Yelp per request
SEARCH_RADIUS = 6000         # Smaller radius for sub-areas (6 km)
LATITUDE = 45.4642           # Center latitude for Milan
LONGITUDE = 9.19             # Center longitude for Milan

def make_request(url, params=None):
    """
    Makes a request to the Yelp API.
    """
    response = requests.get(url, headers=HEADERS, params=params)
    response.raise_for_status()
    return response

def search_businesses(location, term="restaurant", limit=BUSINESSES_PER_REQUEST, offset=0):
    """
    Searches for businesses in a given location with pagination.
    """
    url = "https://api.yelp.com/v3/businesses/search"
    params = {
        "location": location,  # Neighborhood or address
        "term": term,
        "limit": limit,
        "offset": offset
    }
    response = make_request(url, params=params)
    return response.json().get("businesses", [])

# Initialize data storage
data = []

try:
    for neighborhood in NEIGHBORHOODS:  # Loop through each neighborhood
        offset = 0
        while True:  # Keep fetching data until no more results
            print(f"Fetching businesses in {neighborhood} with offset: {offset}...")

            # Fetch businesses using the current offset and location
            businesses = search_businesses(location=neighborhood, term="restaurant", limit=BUSINESSES_PER_REQUEST, offset=offset)

            if not businesses:
                # No more businesses to fetch
                print(f"No more businesses returned for {neighborhood}.")
                break

            for biz in businesses:
                name = biz.get("name", "N/A")
                location_info = biz.get("location", {})
                address = location_info.get("address1", "N/A")
                zipcode = location_info.get("zip_code", "N/A")
                categories = biz.get("categories", [])
                category_list = [cat.get("title", "") for cat in categories if cat.get("title")]
                category_str = ", ".join(category_list) if category_list else "N/A"
                rating = biz.get("rating", "N/A")
                review_count = biz.get("review_count", "N/A")
                price = biz.get("price", "N/A")

                # Extract coordinates
                coordinates = biz.get("coordinates", {})
                latitude = coordinates.get("latitude", None)
                longitude = coordinates.get("longitude", None)

                # Append to data
                data.append({
                    "Neighborhood": neighborhood,
                    "Business Name": name,
                    "Business Address": address,
                    "Categories": category_str,
                    "Average Star Rating": rating,
                    "Review Count": review_count,
                    "Zipcode": zipcode,
                    "Price": price,
                    "Latitude": latitude,
                    "Longitude": longitude
                })

            # Increment offset for the next batch
            offset += len(businesses)

            # Optional: Sleep to respect API rate limits
            time.sleep(0.5)

            # Break if the offset exceeds Yelp's maximum results per query
            if offset >= 240:  # Maximum 240 results per query
                print(f"Reached maximum results for {neighborhood}.")
                break

    # Convert the collected data into a DataFrame
    df = pd.DataFrame(data)

    # Convert DataFrame to GeoDataFrame
    df["geometry"] = df.apply(
        lambda row: Point(row["Longitude"], row["Latitude"]) if row["Longitude"] and row["Latitude"] else None,
        axis=1
    )
    gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")

    # Save to file if needed
    gdf.to_file("milan_restaurants.geojson", driver="GeoJSON")

    # Print summary
    print(gdf.head())
    print(f"Total businesses collected: {len(data)}")

except requests.HTTPError as he:
    print(f"HTTP error occurred: {he}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Fetching businesses in Brera, Milan with offset: 0...
Fetching businesses in Brera, Milan with offset: 48...
Fetching businesses in Brera, Milan with offset: 96...
Fetching businesses in Brera, Milan with offset: 144...
Fetching businesses in Brera, Milan with offset: 192...
Reached maximum results for Brera, Milan.
Fetching businesses in Duomo, Milan with offset: 0...
Fetching businesses in Duomo, Milan with offset: 48...
Fetching businesses in Duomo, Milan with offset: 96...
Fetching businesses in Duomo, Milan with offset: 144...
Fetching businesses in Duomo, Milan with offset: 192...
Reached maximum results for Duomo, Milan.
Fetching businesses in Porta Sempione, Milan with offset: 0...
Fetching businesses in Porta Sempione, Milan with offset: 48...
Fetching businesses in Porta Sempione, Milan with offset: 96...
Fetching businesses in Porta Sempione, Milan with offset: 144...
Fetching businesses in Porta Sempione, Milan with offset: 192...
Reached maximum results for Porta Sempione,

In [26]:
gdf_nodup = gdf.drop_duplicates(subset=['Business Name','Business Address','Categories','Average Star Rating','Review Count', 'Zipcode', 'geometry'])

In [27]:
gdf_nodup

,Neighborhood,Business Name,Business Address,Categories,Average Star Rating,Review Count,Zipcode,Price,Latitude,Longitude,geometry
0,"Brera, Milan",Risoelatte,Via Camperio 6,Italian,4.4,160,20123,€€,45.466417,9.183352,POINT (9.18335 45.46642)
1,"Brera, Milan",Salsamenteria di Parma,Via San Pietro all'Orto 9,Emilian,4.4,180,20121,€€,45.466267,9.195122,POINT (9.19512 45.46627)
2,"Brera, Milan",Nabucco,Via Fiori Chiari 10,"Seafood, Mediterranean, Lumbard",4.3,51,20121,€€€,45.472240,9.186840,POINT (9.18684 45.47224)
3,"Brera, Milan",Settembrini 18,Via Settembrini 18,"Italian, Seafood, Pizza",4.5,559,20124,€€,45.481243,9.205606,POINT (9.20561 45.48124)
4,"Brera, Milan",DeCanto,Piazza della Scala 6,"Brasseries, Cafeteria, Italian",4.4,5,20121,€€,45.467395,9.190173,POINT (9.19017 45.46739)
...,...,...,...,...,...,...,...,...,...,...,...
952,"Guastalla, Milan",Chip & Chic,Galleria Unione 1,"Breakfast & Brunch, Bars",4.0,1,20122,N/A,45.459792,9.196806,POINT (9.19681 45.45979)
953,"Guastalla, Milan",Roqus,Viale Bligny 2,Pizza,4.5,8,20136,€,45.451656,9.187187,POINT (9.18719 45.45166)
954,"Guastalla, Milan",Panigacci,Via Amatore Sciesa 7,"Ligurian, Sandwiches",3.9,7,20135,€,45.461281,9.209530,POINT (9.20953 45.46128)
955,"Guastalla, Milan",Brezzamarina,Viale Umbria 50,"Pizza, Seafood",5.0,1,20137,N/A,45.452880,9.214940,POINT (9.21494 45.45288)


In [28]:
# DEVONO ESSERE LO STESSO TIPO DI COORDINATE
print(gdf.crs)  # Restaurants CRS
print(gdf_quarters.crs)  # Neighborhood polygons CRS


EPSG:4326
EPSG:4326


# Join Spaziale

In [30]:
from geopandas.tools import sjoin

# Filter columns in gdf_quarters
gdf_quarters = gdf_quarters[["name", "ref", "source", "geometry"]]

# Perform the spatial join
joined = sjoin(gdf, gdf_quarters, how="left", predicate="within")

# Check the result
joined.head(10)

,Neighborhood,Business Name,Business Address,Categories,Average Star Rating,Review Count,Zipcode,Price,Latitude,Longitude,geometry,index_right,name,ref,source
0,"Brera, Milan",Risoelatte,Via Camperio 6,Italian,4.4,160,20123,€€,45.466417,9.183352,POINT (9.18335 45.46642),21.0,Duomo,1,Comune di Milano
1,"Brera, Milan",Salsamenteria di Parma,Via San Pietro all'Orto 9,Emilian,4.4,180,20121,€€,45.466267,9.195122,POINT (9.19512 45.46627),21.0,Duomo,1,Comune di Milano
2,"Brera, Milan",Nabucco,Via Fiori Chiari 10,"Seafood, Mediterranean, Lumbard",4.3,51,20121,€€€,45.472240,9.186840,POINT (9.18684 45.47224),9.0,Brera,2,Comune di Milano
3,"Brera, Milan",Settembrini 18,Via Settembrini 18,"Italian, Seafood, Pizza",4.5,559,20124,€€,45.481243,9.205606,POINT (9.20561 45.48124),11.0,Buenos Aires - Venezia,21,Comune di Milano
4,"Brera, Milan",DeCanto,Piazza della Scala 6,"Brasseries, Cafeteria, Italian",4.4,5,20121,€€,45.467395,9.190173,POINT (9.19017 45.46739),21.0,Duomo,1,Comune di Milano
5,"Brera, Milan",Tartufotto,Via Cusani 8,"Italian, Wine Bars, Bistros",4.4,39,20121,€€€,45.468923,9.184549,POINT (9.18455 45.46892),9.0,Brera,2,Comune di Milano
6,"Brera, Milan",Bioesserì,Via Fatebenefratelli 2,"Desserts, Vegetarian, Pizza",4.8,4,20121,€€,45.472900,9.188490,POINT (9.18849 45.4729),9.0,Brera,2,Comune di Milano
7,"Brera, Milan",La Prosciutteria - Brera,Corso Garibaldi 55,"Wine Bars, Tuscan, Sandwiches",4.5,49,20121,€€,45.474320,9.183400,POINT (9.1834 45.47432),9.0,Brera,2,Comune di Milano
8,"Brera, Milan",Le 5 Terre,Via Andrea Appiani 9,"Ligurian, Seafood, Wine Bars",4.9,9,20121,N/A,45.477302,9.193683,POINT (9.19368 45.4773),9.0,Brera,2,Comune di Milano
9,"Brera, Milan",Puglia in Brera,Via San Carpoforo 6,"Apulian, Trattorie",5.0,2,20121,€€€,45.471510,9.184870,POINT (9.18487 45.47151),9.0,Brera,2,Comune di Milano


In [31]:
joined[joined['Business Name']=='Tartufotto'] # giusto

,Neighborhood,Business Name,Business Address,Categories,Average Star Rating,Review Count,Zipcode,Price,Latitude,Longitude,geometry,index_right,name,ref,source
5,"Brera, Milan",Tartufotto,Via Cusani 8,"Italian, Wine Bars, Bistros",4.4,39,20121,€€€,45.468923,9.184549,POINT (9.18455 45.46892),9.0,Brera,2,Comune di Milano
265,"Duomo, Milan",Tartufotto,Via Cusani 8,"Italian, Wine Bars, Bistros",4.4,39,20121,€€€,45.468923,9.184549,POINT (9.18455 45.46892),9.0,Brera,2,Comune di Milano
509,"Porta Sempione, Milan",Tartufotto,Via Cusani 8,"Italian, Wine Bars, Bistros",4.4,39,20121,€€€,45.468923,9.184549,POINT (9.18455 45.46892),9.0,Brera,2,Comune di Milano
792,"Guastalla, Milan",Tartufotto,Via Cusani 8,"Italian, Wine Bars, Bistros",4.4,39,20121,€€€,45.468923,9.184549,POINT (9.18455 45.46892),9.0,Brera,2,Comune di Milano
